In [3]:
# If running in Google Colab, import files
# !pip install numpy pandas scipy matplotlib tensorflow-gpu keras tqdm
try:
    import google.colab
    in_colab = True
except:
    in_colab = False

if in_colab:
    !git clone https://github.com/Hyperparticle/one-pixel-attack-keras.git
    !mv -v one-pixel-attack-keras/* .
    !rm -rf one-pixel-attack-keras

# Python Libraries
%matplotlib inline
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from keras.datasets import cifar10
from keras import backend as K # original
#from kears import keras.backend.tensorflow_backend as K
import tensorflow as tf
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib

# Custom Networks
from networks.lenet import LeNet
from networks.pure_cnn import PureCnn
from networks.network_in_network import NetworkInNetwork
from networks.resnet import ResNet
from networks.densenet import DenseNet
from networks.wide_resnet import WideResNet
from networks.capsnet import CapsNet

# Helper functions
from differential_evolution import differential_evolution
import helper

matplotlib.style.use('ggplot')
np.random.seed(100)

## Imports

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

## Image Perturbation

In [6]:
def perturb_image(xs, img):
    # If this function is passed just one perturbation vector,
    # pack it in a list to keep the computation the same
    if xs.ndim < 2:
        xs = np.array([xs])
    
    # Copy the image n == len(xs) times so that we can 
    # create n new perturbed images
    tile = [len(xs)] + [1]*(xs.ndim+1)
    imgs = np.tile(img, tile)
    # numpy tile(A, repeat_shape)
    # A라는 array가 repeat_shape 모양으로 쌓이도록
    # 추후에 다시 보자..
    
    # Make sure to floor the members of xs as int types
    xs = xs.astype(int)
    
    for x,img in zip(xs, imgs):
        # Split x into an array of 5-tuples (perturbation pixels)
        # i.e., [[x,y,r,g,b], ...]
        pixels = np.split(x, len(x) // 5)
        # pixels = 5-tuple of perturb information (x,y,r,g,b) by kyle
        for pixel in pixels:
            # At each pixel's x,y position, assign its rgb value
            x_pos, y_pos, *rgb = pixel # *rgb = 뒤에 올 (r,g,b) 값을 가리킨다
            img[x_pos, y_pos] = rgb # perturb image by *rgb == (r,g,b) by kyle
    
    return imgs
    # Return value is array of perturbed images by xs
    # For example, if xs is [(16,16,255,255,0), (16,16,0,255,255)]
    # imgs is an array of perturbed image by xs[0], xs[1]. So it's size is 2 by kyle

## Load Model

In [7]:
lenet = LeNet()
resnet = ResNet()

# originally, lenet and resnet

## Uncomment below to load more models to play with. Make sure the model files exist by training or downloading them.

# lenet = LeNet()
pure_cnn = PureCnn()
net_in_net = NetworkInNetwork()
# resnet = ResNet()
densenet = DenseNet()
wide_resnet = WideResNet()
# capsnet = CapsNet()
# models = [lenet, resnet, wide_resnet]

models = [lenet, pure_cnn, net_in_net, resnet, densenet, wide_resnet]

Successfully loaded lenet
Successfully loaded resnet
Successfully loaded pure_cnn
Successfully loaded net_in_net
Successfully loaded densenet
Successfully loaded wide_resnet


In [8]:
def predict_classes(xs, img, target_class, model, minimize=True):
    # Perturb the image with the given pixel(s) x and get the prediction of the model
    imgs_perturbed = perturb_image(xs, img)
    predictions = model.predict(imgs_perturbed)[:,target_class]
    # Target class의 값만 뽑아내는 용(파이썬 문법)
    # This function should always be minimized, so return its complement if needed
    return predictions if minimize else 1 - predictions

## OPA2D ATTACK

In [9]:
# This cell is for OPA2D_Attack.

def Algorithm1(image_id, model, sensitive_list):
    
    #sensitive_pixel = sensitive_pixel_sorting(image_id)
    #print(sensitive_pixel)
    x_success = None
    L_opt = 256

    for pixel in sensitive_list:
        L_opt = 256
        print("pixel coordinate : " + str(pixel[0]) + ", " + str(pixel[1]))
        img = x_test[image_id].copy()
        x_pos = pixel[0]
        y_pos = pixel[1]
        r_xk = img[x_pos][y_pos][0]
        g_xk = img[x_pos][y_pos][1]
        b_xk = img[x_pos][y_pos][2]

        a_max = L_opt-1
        a_min = 1
        x_prime = None
        x = [x_pos, y_pos, r_xk, g_xk, b_xk]
        while a_min <= a_max:
            # difference with pseudo code, we have found an error of code, - -> + (minus to plus)
            a = (a_max + a_min) // 2
            R_xk_bounds = [(bound(r_xk - a), bound(r_xk+a)), (bound(g_xk-a), bound(g_xk+a)), (bound(b_xk-a), bound(b_xk+a))]

            x_prime, success = opa2d_attack(image_id, model, R_xk_bounds, x_pos, y_pos)
            print("alpha : ", a)
            print("alpha_min : ", a_min)
            if success:
                L_opt = dist(x,x_prime)
                a_max = L_opt-1
                x_success = x_prime
            else:
                a_min = a_min + a + 1

    if(L_opt < 256):
        return (x_success,"attack success")
    else:
        return (x_success,"attack failed")
    
def dist(x, x_prime):
    return max(abs(x[2] - x_prime[2]), abs(x[3] - x_prime[3]), abs(x[4] - x_prime[4]))

def bound(num):
    return max(0, min(255,num))

def opa2d_attack(img_id, model, R_xk_bounds, x_pos, y_pos, target=None, pixel_count=1, maxiter=5, popsize=50, verbose=False):
    # Change the target class based on whether this is a targeted attack or not
    targeted_attack = target is not None
    target_class = target if targeted_attack else y_test[img_id, 0]
    
    # Define bounds for a flat vector of x,y,r,g,b values
    # For more pixels, repeat this layout
    bounds = [(x_pos, x_pos), (y_pos, y_pos), R_xk_bounds[0], R_xk_bounds[1], R_xk_bounds[2]] * pixel_count

    # Population multiplier, in terms of the size of the perturbation vector x
    popmul = max(1, popsize // len(bounds))
    
    # Format the predict/callback functions for the differential evolution algorithm
    def predict_fn(xs):
        return predict_classes(xs, x_test[img_id], target_class, 
                               model, target is None)
    
    def callback_fn(x, convergence):
        return attack_success(x, x_test[img_id], target_class, 
                              model, targeted_attack, verbose)
    
    # Call Scipy's Implementation of Differential Evolution
    attack_result = differential_evolution(
        predict_fn, bounds, maxiter=maxiter, popsize=popmul,
        recombination=1, atol=-1, callback=callback_fn, polish=False)
    
    # Calculate some useful statistics to return from this function
    attack_image = perturb_image(attack_result.x, x_test[img_id])[0]
    predicted_probs = model.predict_one(attack_image)
    predicted_class = np.argmax(predicted_probs)
    actual_class = y_test[img_id, 0]
    # helper.plot_image(attack_image, actual_class, class_names, predicted_class)

    success = predicted_class != actual_class
    
    return [(attack_result.x[0], attack_result.x[1], attack_result.x[2], attack_result.x[3], attack_result.x[4]), success]

In [10]:
def sensitive_pixel_sorting(image_id):
    img = x_test[image_id].copy()
    target_class = y_test[image_id, 0]

    diff_list = [[0,0,0]]

    prior_confidence =  model.predict_one(img)[target_class]

    rgb = [0, 0, 0]
    
    sensitive_list_size = 30

    for x in range(32):
        for y in range(32):

          for i in range(3):
              if img[x][y][i] >= 128:
                rgb[i] = 0
              else:
                rgb[i] = 255

          pixel = np.array([x, y,  rgb[0], rgb[1], rgb[2]])
          perturbed_image = perturb_image(pixel, img)
          
          after_confidence =  model.predict_one(perturbed_image)[target_class]
          diff_confidence = abs(after_confidence - prior_confidence)

          if diff_confidence > 0:
              for diff_index in range(len(diff_list)):
                  if diff_confidence >= diff_list[diff_index][2]:
                      diff_list.insert(diff_index, [x, y, diff_confidence])
                      if (len(diff_list) > sensitive_list_size):
                          diff_list.pop()
                      break
    return diff_list

In [11]:
def attack_success(x, img, target_class, model, targeted_attack=False, verbose=False):
    # Perturb the image with the given pixel(s) and get the prediction of the model
    attack_image = perturb_image(x, img)

    confidence = model.predict(attack_image)[0] # attack_image가 배열이다. 그 중 하나를 리턴.
    predicted_class = np.argmax(confidence) # 그 중 최댓값을 고른다..
    
    # If the prediction is what we want (misclassification or 
    # targeted classification), return True
    if verbose:
        print('Confidence:', confidence[target_class])
    if ((targeted_attack and predicted_class == target_class) or
        (not targeted_attack and predicted_class != target_class)):
        return True
    # NOTE: return None otherwise (not False), due to how Scipy handles its callback function

In [12]:
image_id_list = [16,20, 25, 87, 384, 520]
models = [lenet, pure_cnn, net_in_net, resnet, densenet, wide_resnet]
image_id = 16
pixels = 1 # Number of pixels to attack
model = resnet
# for i in range(len(image_id_list)):
time_list = [[0]*len(models)] 
for i in range(3):
    for j in range(len(models)):
        model = models[j]
        print(models[j])
        sen_list = sensitive_pixel_sorting(image_id_list[i])
        # model = models[j]
        # print(sen_list)
        print(len(sen_list))
        list_num = 0
        start = time.time()
        # _ = my_attack_3(image_id, model, pixel_count=pixels, maxiter = 5, popsize = 50, verbose = True)
        
        _ = Algorithm1(image_id, models[j], sen_list)
        end = time.time()
        print(f"{end - start:.6f} sec")
        time_list.append(end-start)
    print("image_id : ",image_id)
    print(time_list)
    time_list.clear()
    

30
pixel coordinate : 24, 14
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 23, 11
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 24, 13
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 22, 10
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 25, 8
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 26, 8
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 25, 7
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 24, 24
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 23, 24
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 22, 24
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 24, 10
alpha :  128
alpha_min :  1
alpha :  192
alpha_min :  130
pixel coordinate : 26, 6
alpha :  128
alpha_min :  1
alpha :  192
alpha_min 